#### Ingestion del archivo production_country

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo Json usando "DataFrameReader" en Spark

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

In [0]:
production_country_schema = StructType(fields = [
  StructField("movieId", IntegerType(), False),
  StructField("countryId", IntegerType(),False)
])

In [0]:
production_country_df = spark.read.option("multiline","True").schema(production_country_schema) \
                                                        .json(f"{bronze_folder_path}/{v_file_date}/production_country")

#### Paso 2 - Agregar los campos faltnates y cambiar de nombre
##### 1. Cambiar el nombre de "movieId" a "movie_id"
##### 2. Cambiar el nombre de "countryId" a "country_id"
##### 3. Agregar "enviroment" y Agregar "ingestion_date"

In [0]:
from pyspark.sql.functions import lit, current_timestamp,col

In [0]:
final_production_country = add_ingestion_date(production_country_df).withColumnRenamed("movieId","movie_id") \
                                .withColumnRenamed("countryId","country_id") \
                                .withColumn("enviromen",lit(v_enviroment)) \
                                .withColumn("file_date",lit(v_file_date))

#### Paso 3 - Guardar el DF en parquet

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date'
merge_delta_lake(final_production_country, "movie_silver", "production_country", silver_foler_path, merge_condition, "file_date")


#overwrite_partition(final_production_country,"movie_silver","production_country","file_date")

In [0]:
%sql
SELECT COUNT(1), file_date
FROM movie_silver.production_country
GROUP BY file_date

In [0]:
dbutils.notebook.exit("Success!")